In [63]:
import os
import openai
import yaml

import re


import json
import re

# Load API key from a YAML configuration file
with open("config.yaml") as f:
 config_yaml = yaml.load(f, Loader=yaml.FullLoader)

# Initialize the OpenAI client with the API key
client = openai.OpenAI(api_key=config_yaml['token'])

# Set the model name
MODEL = "gpt-4o-mini"

def llm(prompt, stop=["\n"]):
    # Prepare the dialog for the API request
    dialogs = [{"role": "user", "content": prompt}]

    completion = client.chat.completions.create(
        model=MODEL,
        messages=dialogs
    )
    return completion.choices[0].message.content

In [2]:
def generate_comparison_feedback_prompt(description1, description2):
    prompt = f"""
<TASK>
You are tasked with analyzing two motion descriptions, `description1` (generated by a model) and `description2` (a gold-standard or reference description). Your goal is to carefully compare the two descriptions for each of the following body parts: spine, left arm, right arm, left leg, and right leg. For each body part, identify any discrepancies between the two descriptions and provide a single piece of feedback to help improve `description1` so it more closely matches `description2`. It is better to use standard command words from the Standard_Manual.

<BODY_PARTS>
- spine
- Left Arm
- Right Arm
- Left Leg
- Right Leg
</BODY_PARTS>

<description1>
{description1}
</description1>

<description2>
{description2}
</description2>

<Standard_Manual>
**Standard Manual for Human Motion Description**

This manual aims to standardize the language used by models when describing human poses and motions, making it more precise and specific while avoiding vague and abstract expressions. By employing a unified set of terms, phrases, and descriptive standards, it ensures that the content generated by the model is clear and consistent, making it easier for readers to understand and apply.

---

### **1. Introduction**

In the fields of artificial intelligence and computer vision, accurately describing human motion is crucial for the training and application of models. This manual provides a comprehensive and detailed set of guidelines to assist models in generating precise human motion descriptions, avoiding ambiguity and vagueness.

### **2. Basic Principles**

- **Precision**: Descriptions should be as specific as possible, providing clear angles, directions, positions, and motion details.
- **Consistency**: Use a uniform set of terms and phrases to ensure the descriptions are coherent and comprehensible.
- **Objectivity**: Avoid subjective evaluations or emotional tones, focusing on the statement of objective facts.
- **Conciseness**: Expressions should be brief and to the point, highlighting key information while avoiding redundancy.

### **3. Description Standards**

#### **3.1 Angle and Direction Descriptions**

**Angle Categories:**

- **Vertical**: Angle ≤ 10°
  - **Example**: *“The arm is raised vertically.”*
- **Near Vertical**: 10° < Angle ≤ 30°
  - **Example**: *“The thigh is nearly vertical to the ground.”*
- **Diagonal**: 30° < Angle ≤ 60°
  - **Example**: *“The arm extends at a 45-degree angle to the front side.”*
- **Near Horizontal**: 60° < Angle ≤ 80°
  - **Example**: *“The lower leg is nearly horizontal.”*
- **Horizontal**: Angle > 80°
  - **Example**: *“The arms are extended horizontally to the sides.”*

**Direction Descriptions:**

- **Front, Back, Left, Right, Up, Down**
  - **Example**: *“The head turns to the left.”*, *“The leg steps forward.”*
- **Inner, Outer**
  - **Example**: *“The palm rotates inward.”*, *“The toes point outward.”*
- **Clockwise, Counterclockwise**
  - **Example**: *“The arm rotates in a clockwise direction.”*

#### **3.2 Motion Type Descriptions**

- **Static Motion**: Maintaining a pose without movement.
  - **Example**: *“He stands still with his hands by his sides.”*
- **Continuous Motion**: Motion ongoing without interruption.
  - **Example**: *“She continues to run with a steady pace.”*
- **Repetitive Motion**: Actions that are repeated in a regular pattern.
  - **Example**: *“He jumps up and down repeatedly.”*
- **Single Motion**: An action that is completed once.
  - **Example**: *“She raises her arm and waves.”*

#### **3.3 Motion Amplitude and Degree Descriptions**

- **Complete**: The action reaches its maximum range.
  - **Example**: *“The knee is bent completely to bring the calf against the thigh.”*
- **Large Amplitude**: The action is close to its maximum range.
  - **Example**: *“He swings his arm widely.”*
- **Medium Amplitude**: The action reaches a general range.
  - **Example**: *“She raises her leg at a moderate height.”*
- **Small Amplitude**: The action has a small range.
  - **Example**: *“He slightly shakes his head.”*
- **Slightly**: The action amplitude is minimal, with little change.
  - **Example**: *“She slightly lifts her chin.”*

#### **3.4 Time and Frequency Descriptions**

- **One-time**: The action occurs only once.
  - **Example**: *“He presses the button.”*
- **Continuous**: The action is performed uninterrupted over time.
  - **Example**: *“She is continuously stretching her body.”*
- **Intermittent**: The action occurs at intervals.
  - **Example**: *“He intermittently types on the keyboard.”*
- **Frequent**: The action repeats multiple times in a short period.
  - **Example**: *“She blinks frequently.”*
- **Slow**: The action is performed at a slow pace.
  - **Example**: *“He slowly raises his arm.”*
- **Fast**: The action is performed at a fast pace.
  - **Example**: *“She quickly turns around.”*

#### **3.5 Position and Spatial Relationship Descriptions**

- **Contact**: Between body parts or with an object.
  - **Example**: *“The palm is pressed against the wall.”*
- **Distance**: Describing the space between two body parts or objects.
  - **Close**: Distance ≤ 0.2 meters
    - **Example**: *“The feet are close together.”*
  - **Apart**: 0.2 meters < Distance ≤ 0.5 meters
    - **Example**: *“The hands are shoulder-width apart.”*
  - **Far Apart**: Distance > 0.5 meters
    - **Example**: *“The arms are extended far from the body.”*
- **Height**: Relative to the ground or other reference points.
  - **Below, Equal to, Above**
    - **Example**: *“The shoulder height is equal to the table height.”*

### **4. Terms and Definitions**

#### **4.1 Body Part Names**

```
<BODY_PARTS>
- spine
- Left Arm
- Right Arm
- Left Leg
- Right Leg
</BODY_PARTS>
```

#### **4.2 Action Verbs**

- **Basic Actions**: Raise, lower, bend, extend, rotate, kick, grasp, push, pull, jump, squat, stand, sit, lie
- **Compound Actions**: Step over, squat down, flip forward, flip backward, kick leg, stretch, twist

#### **4.3 Directional Terms**

- **Horizontal Directions**: Front, back, left, right
- **Vertical Directions**: Up, down
- **Spatial Relationships**: Inner, outer, front, back, side, opposite

#### **4.4 Angle and Amplitude Terms**

- **Angle Unit**: Degrees (°)
- **Angle Descriptions**: Vertical, horizontal, diagonal, right angle (90°)
- **Amplitude Descriptions**: Complete, large amplitude, medium amplitude, small amplitude, slight



### **7. Notes**

#### **7.1 Avoid Vague and Subjective Descriptions**

- **Non-standard**: *“He moves in a strange way.”*
- **Standard**: *“He crosses his left foot over his right foot and leans to the right.”*

#### **7.2 Ensure Consistency and Precision**

- Always use the terms and standards outlined in this manual, avoiding different terms for the same entity.



### **8. Appendix**

#### **8.1 Common Angle and Distance Reference Table**

| Angle Description | Range     |
| ----------------- | --------- |
| Vertical          | 0° - 10°  |
| Near Vertical     | 10° - 30° |
| Diagonal          | 30° - 60° |
| Near Horizontal   | 60° - 80° |
| Horizontal        | 80° - 90° |

| Distance Description | Range                   |
| -------------------- | ----------------------- |
| Close                | ≤ 0.2 meters            |
| Apart                | 0.2 meters - 0.5 meters |
| Far Apart            | > 0.5 meters            |

 

</Standard_Manual>

<INSTRUCTIONS>

Editing command: to help improve `description1` so it more closely matches `description2`.
1. **Comparison**:
    - Compare the descriptions for each body part mentioned above. Focus on specific aspects of the motion, such as:
        - **Movement**: How the body part is moving (e.g., raised, lowered, rotated).
        - **Direction**: In which direction the movement occurs (e.g., upward, downward, forward, backward).
        - **Intensity**: Is the movement fast or slow? Sudden or smooth?
        - **Positioning**: The final position of the body part (e.g., shoulder height, bent at the knee).
    - If a body part is mentioned in `description2` but not in `description1`, note that as a discrepancy.

2. **Feedback**:
    - For each body part, if a discrepancy is found, generate **one** piece of feedback that includes:
        - **Body part**: Identify which body part the feedback is for (e.g., left arm, right leg, spine).
        - **Issue**: Briefly describe the difference between `description1` and `description2` for this body part.
        - **Suggested Change**: Provide a clear, actionable suggestion to adjust `description1` to match `description2`. Be specific about how the movement should be changed.
    - Ensure that only one piece of feedback is given for each body part, even if there are multiple issues. Prioritize the most significant discrepancy.

3. **Output**:
    - Format the feedback as follows:
    {{
        "body part": "spine",
        "issue": "[description of discrepancy in spine movement]",
        "editing command": "The man..../ The man's spine ... [specific suggestion to correct the spine movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "left arm",
        "issue": "[description of discrepancy in left arm movement]",
        "suggestion": "The man..../ The man's left arm ... [specific suggestion to correct the left arm movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "right arm",
        "issue": "[description of discrepancy in right arm movement]",
        "suggestion": "The man..../ The man's right arm ... [specific suggestion to correct the right arm movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "left leg",
        "issue": "[description of discrepancy in left leg movement]",
        "suggestion": "The man..../ The man's left leg ... [specific suggestion to correct the left leg movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "right leg",
        "issue": "[description of discrepancy in right leg movement]",
        "suggestion": "The man..../ The man's right leg ... [specific suggestion to correct the right leg movement,should use Standard_Manual style of words]"
    }}

4. **Exclusions**:
    - Do not analyze facial expressions, emotions, or any abstract descriptions.
    - Focus only on the physical actions of the body parts mentioned in `<BODY_PARTS>`.

<END_SIGNAL>
When you have finished generating the feedback, end the output with '<FEEDBACKEND>'.
</END_SIGNAL>
</INSTRUCTIONS>

Now, compare the two descriptions and generate feedback.
    """
    return prompt



In [66]:
def decide_which_part_needs_editing_prompt(description1, description2):
    prompt = f"""
<TASK>
You are tasked with determining which parts of the body need editing in `description1` (generated by a model) compared to `description2` (a gold-standard or reference description). Your goal is to carefully compare the two descriptions for each of the following body parts: spine, left arm, right arm, left leg, and right leg.

For each body part, identify discrepancies between the two descriptions and provide a boolean value indicating whether `description1` needs editing for that body part to match `description2`.

<BODY_PARTS>
- Spine
- Left Arm
- Right Arm
- Left Leg
- Right Leg
</BODY_PARTS>

<description1>
{description1}
</description1>

<description2>
{description2}
</description2>

<OUTPUT>
{{
  "action_description": "...",
  "gold_standard": "...",
  "output": {{
  "Spine": true/false,
  "Left_Arm": true/false,
  "Right_Arm": true/false,
  "Left_Leg": true/false,
  "Right_Leg": true/false,
    }}
}}

</OUTPUT>

<example>
    description1 = 
    A person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.
    
    description2 =  
    The man is walking clockwise in a circle while holding something up to his ear with his left arm.
    
    
    <OUTPUT>
{{
  "action_description": "A person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.",
  "gold_standard": "The man is walking clockwise in a circle while holding something up to his ear with his left arm.",
  "output": {{
    "Spine": false,
    "Left Arm": true,
    "Right Arm": true,
    "Left Leg": false,
    "Right Leg": false,
  }}
}}

    </OUTPUT>
</example>

Strictly output in JSON format. No additional explanations or elaborations are allowed.

Specifically, after "Right_Leg": false/true, a comma is expected before closing the curly braces (}}). This issue is causing the JSON decoding process to fail. MAKE sure the json is correct ,I don't want to see sth like"JSONDecodeError: Expecting ',' delimiter: line 10 column 4 (char 378)
Comparison Feedback Results saved to "
Specifically, after "Right_Leg": false/true, a comma is expected before closing the curly braces (}}). This issue is causing the JSON decoding process to fail. MAKE sure the json is correct ,I don't want to see sth like"JSONDecodeError: Expecting ',' delimiter: line 10 column 4 (char 378)
Comparison Feedback Results saved to "
Specifically, after "Right_Leg": false/true, a comma is expected before closing the curly braces (}}). This issue is causing the JSON decoding process to fail. MAKE sure the json is correct ,I don't want to see sth like"JSONDecodeError: Expecting ',' delimiter: line 10 column 4 (char 378)
Comparison Feedback Results saved to "
Specifically, after "Right_Leg": false/true, a comma is expected before closing the curly braces (}}). This issue is causing the JSON decoding process to fail. MAKE sure the json is correct ,I don't want to see sth like"JSONDecodeError: Expecting ',' delimiter: line 10 column 4 (char 378)
Comparison Feedback Results saved to "
<END_SIGNAL>
When you have finished generating the feedback, end the output with '<FEEDBACKEND>'.
</END_SIGNAL>

"""
    return prompt



In [26]:
# import json
# import re
# 
# def decide_which_part_needs_editing(description1, description2):
#     # 生成对比反馈的 prompt
#  
#     comparison_prompt = decide_which_part_needs_editing_prompt(description1, description2)
#  
#     comparison_feedback = llm(comparison_prompt, stop=["<END_FEEDBACK>"]).split("<END_FEEDBACK>")[0].strip()
#     print("comparison_feedback",comparison_feedback)
# 
#     json_objects = re.findall(r'\{[^}]+\}', comparison_feedback)
#     feedback_results = [json.loads(obj) for obj in json_objects]
#     
#     # 保存到文件中
#     output_filename = "decide_which_part_needs_editing_prompt.json"
#     with open(output_filename, "w") as file:
#         json.dump(feedback_results, file, ensure_ascii=False, indent=2)
#     
#     # 打印输出
#     print(f"Comparison Feedback Results saved to {output_filename}")
#     print(json.dumps(feedback_results, ensure_ascii=False, indent=2))
#     
#     # 返回生成的反馈结果
#     return feedback_results
# 


In [28]:
import json
import re

def test_generate_comparison_feedback(description1, description2):
    # Generate comparison feedback prompt
    comparison_prompt = generate_comparison_feedback_prompt(description1, description2)
    
    # Get feedback from LLM and remove "<FEEDBACKEND>" tag
    comparison_feedback = llm(comparison_prompt, stop=["<END_FEEDBACK>"]).replace("<FEEDBACKEND>", "").strip()

    # Check if the feedback is valid JSON
    try:
        # Convert feedback directly to a JSON object
        feedback_results = json.loads(comparison_feedback)
    except json.JSONDecodeError:
        # If not directly valid JSON, try to find and extract JSON-like objects using regex
        json_objects = re.findall(r'\{[^}]+\}', comparison_feedback)
        feedback_results = [json.loads(obj) for obj in json_objects]

    # Save the results to a JSON file
    output_filename = "comparison_feedback_results.json"
    with open(output_filename, "w") as file:
        json.dump(feedback_results, file, ensure_ascii=False, indent=2)

    # Print output
    print(f"Comparison Feedback Results saved to {output_filename}")
    print(json.dumps(feedback_results, ensure_ascii=False, indent=2))

    # Return the generated feedback results
    return feedback_results


In [ ]:
# 金标准和带偏差的动作描述


# 第二个金标准和带偏差的动作描述
description12= """
a person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.
"""

description22 = """
he man is walking clockwise in a circle while holding something up to his ear with his left arm.
"""



# 测试生成反馈
test_generate_comparison_feedback(description12, description22)
#`description1` (generated by a model) and `description2` (a gold-standard or reference description)


In [67]:
import json
import re

def decide_which_part_needs_editing(description1, description2):
    # Generate comparison feedback prompt
    comparison_prompt = decide_which_part_needs_editing_prompt(description1, description2)
    
    # Get feedback from LLM and remove "<FEEDBACKEND>" tag
    comparison_feedback = llm(comparison_prompt, stop=["<END_FEEDBACK>"]).replace("<FEEDBACKEND>", "").strip()

    print(f"Raw feedback: {comparison_feedback}\n")
    
    # Initialize an empty list to store valid JSON objects
    feedback_results = []

    # Extract the JSON content from feedback
    json_start = comparison_feedback.find('{')
    json_end = comparison_feedback.rfind('}') + 1  # Include the closing brace

    if json_start != -1 and json_end != -1:
        json_str = comparison_feedback[json_start:json_end]
        print(f"Extracted JSON string: {json_str}\n")
        try:
            parsed_obj = json.loads(json_str)
            feedback_results.append(parsed_obj)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    else:
        print("No JSON content found in the feedback.")

    # Save the results to a JSON file
    output_filename = "comparison_feedback_results.json"
    with open(output_filename, "w") as file:
        json.dump(feedback_results, file, ensure_ascii=False, indent=2)
    
    # Print output
    print(f"Comparison Feedback Results saved to {output_filename}")
    print(json.dumps(feedback_results, ensure_ascii=False, indent=2))
    
    # Return the generated feedback results
    return feedback_results

# Test the function with sample inputs
description1 = "A person walks in a clockwise circle, while holding their right hand & arm up throughout the motion."
description2 = "The man is walking clockwise in a circle while holding something up to his ear with his left arm."


# Run the test function
decide_which_part_needs_editing(description1, description2)


Raw feedback: ```json
{
  "action_description": "A person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.",
  "gold_standard": "The man is walking clockwise in a circle while holding something up to his ear with his left arm.",
  "output": {
    "Spine": false,
    "Left_Arm": true,
    "Right_Arm": true,
    "Left_Leg": false,
    "Right_Leg": false
  }
}
```

Extracted JSON string: {
  "action_description": "A person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.",
  "gold_standard": "The man is walking clockwise in a circle while holding something up to his ear with his left arm.",
  "output": {
    "Spine": false,
    "Left_Arm": true,
    "Right_Arm": true,
    "Left_Leg": false,
    "Right_Leg": false
  }
}

Comparison Feedback Results saved to comparison_feedback_results.json
[
  {
    "action_description": "A person walks in a clockwise circle, while holding their right hand & arm up throug

[{'action_description': 'A person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.',
  'gold_standard': 'The man is walking clockwise in a circle while holding something up to his ear with his left arm.',
  'output': {'Spine': False,
   'Left_Arm': True,
   'Right_Arm': True,
   'Left_Leg': False,
   'Right_Leg': False}}]

In [60]:
decide_which_part_needs_editing(description12, description22)

{
  "action_description": "a person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.",
  "gold_standard": "he man is walking clockwise in a circle while holding something up to his ear with his left arm.",
  "output": {
    "Spine": false,
    "Left_Arm": true,
    "Right_Arm": true,
    "Left_Leg": false,
    "Right_Leg": false
  }
}
Raw feedback: {
  "action_description": "a person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.",
  "gold_standard": "he man is walking clockwise in a circle while holding something up to his ear with his left arm.",
  "output": {
    "Spine": false,
    "Left_Arm": true,
    "Right_Arm": true,
    "Left_Leg": false,
    "Right_Leg": false
  }
}
Extracted JSON object: {
  "action_description": "a person walks in a clockwise circle, while holding their right hand & arm up throughout the motion.",
  "gold_standard": "he man is walking clockwise in a circle while holding

[]